In [4]:
import pandas as pd
import numpy as np
import openpyxl

temp = pd.read_excel('data/template_수정본.xlsx', engine='openpyxl')
data_2018 = pd.read_excel('data/실업급여_지급현황(월)_2018.xlsx', engine='openpyxl')

In [10]:
(data_2018['직종_중분류'] == '2018직종_정보통신 연구개발직 및 공학기술직').sum()

117

In [13]:
temp[['년/월', '시도', '산업_대분류', '실업급여 수급자 수']]

,년/월,시도,산업_대분류,실업급여 수급자 수
0,201801,서울,"농업, 임업 및 어업",NaN
1,201801,서울,광업,NaN
2,201801,서울,제조업,NaN
3,201801,서울,"전기, 가스, 증기 및 공기조절 공급업",NaN
4,201801,서울,"수도, 하수 및 폐기물 처리, 원료 재생업",NaN
...,...,...,...,...
23455,202212,제주,"협회 및 단체, 수리 및 기타 개인 서비스업",NaN
23456,202212,제주,가구 내 고용활동 및 달리 분류되지 않은 자가소비 생산활동,NaN
23457,202212,제주,국제 및 외국기관,NaN
23458,202212,제주,분류불능,NaN


In [67]:
def pre_df(df, temp):
    df.fillna( method='pad', inplace=True)
    target = df.loc[df['직종_중분류'] == '2018직종_정보통신 연구개발직 및 공학기술직'].reset_index(drop=True)
    target['마감년월'] = target['마감년월'].str.replace('년', '', regex=True).replace('월', '', regex=True)
    target['산업_대분류'] = target['산업_대분류'].str.replace('10차_', '', regex=True)
    del target['직종_중분류']
    target = target[target['(거주지)시도'] != '분류불능']
    region = {'서울특별시':'서울', '부산광역시':'부산', '인천광역시':'인천', '광주광역시':'광주', '대구광역시':'대구', '대전광역시':'대전',
              '울산광역시':'울산', '세종특별자치시':'세종', '경기도':'경기', '강원특별자치도':'강원', '충청남도':'충남', '충청북도':'충북',
              '전라남도':'전남', '전라북도':'전북', '경상남도':'경남', '경상북도':'경북', '제주도':'제주'}
    for k, v in region.items():
        target['(거주지)시도'] = target['(거주지)시도'].str.replace(k, v)
    return target

def merge_file(year):
    import pandas as pd
    import numpy as np
    import openpyxl
    
    df = pd.read_excel(f'data/실업급여_지급현황(월)_{year}.xlsx', engine='openpyxl')
    temp = pd.read_excel('data/template_수정본.xlsx', engine='openpyxl')
    temp = temp[['년/월', '시도', '산업_대분류', '실업급여 수급자 수']]

    import warnings
    warnings.filterwarnings('ignore')

    df = pre_df(df, temp)
    df.columns = temp.columns

    temp['년/월'] = temp['년/월'].astype('str')
    temp['실업급여 수급자 수'].fillna(0, inplace=True)

    temp['실업급여 수급자 수'] = temp['실업급여 수급자 수'].astype('int64')

    temp_year = temp.loc[temp['년/월'].str.startswith(f'{year}')]

    merged = pd.merge(temp_year, df, how='left', on=['년/월', '시도', '산업_대분류'])

    merged.drop(columns=['실업급여 수급자 수_x'], inplace=True)

    merged.columns = ['년/월', '시도', '산업_대분류', '실업급여 수급자 수']

    merged.fillna(0, inplace=True)

    # merged.to_csv(f'data/구인구직취업현황_{year}_merged.csv', index=False)
    
    return merged

In [70]:
df_2018 = merge_file(2018)
df_2019 = merge_file(2019)
df_2020 = merge_file(2020)
df_2021 = merge_file(2021)
df_2022 = merge_file(2022)

dfs = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022], axis=0, ignore_index=True)
print(len(temp), len(dfs))

23460 23460


In [72]:
dfs.to_csv('data/실업급여수급자수_merge.csv', index=False, encoding='utf-8')